#FUNCTIONS TO NORMALIZE 

In [0]:
from pyspark.sql.functions import col, when, lower, lit, to_timestamp, try_to_timestamp, coalesce, year, month, expr, format_string, min as fmin, max as fmax
from pyspark.sql.types import *

##### CHECK IF FORMAT IS NEW #####
def new_format(df):
    return "member_casual" in df.columns

##### NORMALIZE OLD FORMAT #####
def normalize_old(df, year_dir):

    ALIASES = {
        "starttime": "started_at",
        "stoptime": "ended_at",
        "start station name": "start_station_name",
        "start station id": "start_station_id",
        "end station name": "end_station_name",
        "end station id": "end_station_id",
        "start station latitude": "start_lat",
        "start station longitude": "start_lng",
        "end station latitude": "end_lat",
        "end station longitude": "end_lng",
        "usertype": "member_casual"
    }

    VALUE_MAP = {
        "subscriber": "member",
        "customer": "casual"
    }

    # MAPPING COLUMN NAMES
    for old_column, new_column in ALIASES.items():
        if old_column in df.columns:
            df = df.withColumnRenamed(old_column, new_column)
        else: print(f"MAPPING COLUMN NAMES ERROR: Couldn't find column: {old_column}")
    print("column mapping finished")
    
    # MAPPING VALUES MEMBER_CASUAL
    if "member_casual" in df.columns:
        for old_value, new_value in VALUE_MAP.items():
            df = df.withColumn(
                "member_casual",
                when(lower(col("member_casual")) == old_value, new_value)
                .otherwise(col("member_casual"))
            )
        print("member_casual mapping finished")
    else: print(f"MAPPING VALUES MEMBER_CASUAL ERROR: Couldn't find column: member_casual")

    print("(OLD) Standardizing column names and values...")
    df = (
        date_parser(df)
        .withColumn("ride_id", lit(None))
        .withColumn("rideable_type", lit(None))
        .withColumn("duration_sec", (col("ended_at").cast("long") - col("started_at").cast("long")))
        .withColumn("year", year(col("ended_at")))
        .withColumn("month", format_string("%02d", month(col("ended_at"))))
        .select(
            "ride_id",
            "rideable_type",
            "duration_sec",
            "started_at",
            "ended_at",
            "start_station_name",
            "start_station_id",
            "end_station_name",
            "end_station_id",
            "start_lat",
            "start_lng",
            "end_lat",
            "end_lng",
            "member_casual",
            "year",
            "month"
        )
    )
    print("Standardizing column names and values finished")
    return df

##### NORMALIZE NEW FORMAT #####
def normalize_new(df, year_dir):
    print("(NEW) Standardizing column names and values...")
    df = (
        date_parser(df)
        .withColumn("duration_sec", (col("ended_at").cast("long") - col("started_at").cast("long")))
        .withColumn("year", year(col("ended_at")))
        .withColumn("month", format_string("%02d", month(col("ended_at"))))
        .select(
            "ride_id",
            "rideable_type",
            "duration_sec",
            "started_at",
            "ended_at",
            "start_station_name",
            "start_station_id",
            "end_station_name",
            "end_station_id",
            "start_lat",
            "start_lng",
            "end_lat",
            "end_lng",
            "member_casual",
            "year",
            "month"
        )
    )
    print("Standardizing column names and values finished")
    return df

##### CONVERT TO SILVER FORMAT #####
def to_silver(df):
    print("Converting to silver format...")
    SILVER_SCHEMA = StructType([
        StructField("ride_id",              StringType(),       True),
        StructField("rideable_type",        StringType(),       True),
        StructField("duration_sec",         IntegerType(),      True),
        StructField("started_at",           TimestampType(),    True), # starttime
        StructField("ended_at",             TimestampType(),    True), # stoptime
        StructField("start_station_name",   StringType(),       True), # start station name
        StructField("start_station_id",     StringType(),       True), # start station id
        StructField("end_station_name",     StringType(),       True), # end station name
        StructField("end_station_id",       StringType(),       True), # end station id
        StructField("start_lat",            DoubleType(),       True), # start station latitude
        StructField("start_lng",            DoubleType(),       True), # start station longitude
        StructField("end_lat",              DoubleType(),       True), # end station latitude
        StructField("end_lng",              DoubleType(),       True), # end station longitude
        StructField("member_casual",        StringType(),       True), # usertype (values: subscriber, customer)
        StructField("year",                 IntegerType(),      True)
        #StructField("bikeid", IntegerType(), True), -bikeid
        #StructField("birth year", IntegerType(), True), -birthyear
        #StructField("gender", IntegerType(), True) -gender
    ])

    silver_cols = [f.name for f in SILVER_SCHEMA.fields]
    silver_types = {f.name: f.dataType for f in SILVER_SCHEMA.fields}
    print("Silver format conversion complete.")
    return df.select(
        [col(c).cast(silver_types[c]) for c in silver_cols]
    )

def date_parser(df):
    print("Parsing date...")
    df = (
        df
        .withColumn(
            "started_at",
            coalesce(
                try_to_timestamp(col("started_at"), lit("yyyy-MM-dd HH:mm:ss.SSSS")),
                try_to_timestamp(col("started_at"), lit("yyyy-MM-dd HH:mm:ss.SSS")),
                try_to_timestamp(col("started_at"), lit("yyyy-MM-dd HH:mm:ss")),
                try_to_timestamp(col("started_at"), lit("M/d/yyyy HH:mm:ss")),
                try_to_timestamp(col("started_at"), lit("M/d/yyyy H:mm:ss")),
                try_to_timestamp(col("started_at"), lit("M/d/yyyy HH:mm")),
                try_to_timestamp(col("started_at"), lit("M/d/yyyy H:mm")) 
            )
        )
        .withColumn(
            "ended_at",
            coalesce(
                try_to_timestamp(col("ended_at"), lit("yyyy-MM-dd HH:mm:ss.SSSS")),
                try_to_timestamp(col("ended_at"), lit("yyyy-MM-dd HH:mm:ss.SSS")),
                try_to_timestamp(col("ended_at"), lit("yyyy-MM-dd HH:mm:ss")),
                try_to_timestamp(col("ended_at"), lit("M/d/yyyy HH:mm:ss")),
                try_to_timestamp(col("ended_at"), lit("M/d/yyyy H:mm:ss")),
                try_to_timestamp(col("ended_at"), lit("M/d/yyyy HH:mm")),
                try_to_timestamp(col("ended_at"), lit("M/d/yyyy H:mm")) 
            )
        )
    )

    # bad_rows = df.where(
    #     (col("started_at").isNull() | col("ended_at").isNull())
    # ).count()
    # if bad_rows > 0:
    #     display(
    #         df.where(
    #             (col("started_at").isNull() | col("ended_at").isNull())
    #         )
    #     )
    #     raise Exception(f"Parser error: {bad_rows} bad records")

    ymin, ymax = df.select(
        fmin(year(col("ended_at"))),
        fmax(year(col("ended_at")))
    ).first()

    print(f"Parsing complete. Year range: {ymin} – {ymax}")
    return df



#EXECUTION

In [0]:
ADLS_BRONZE = "abfss://citibike@databricksjm.dfs.core.windows.net/bronze"
ADLS_SILVER = "abfss://citibike@databricksjm.dfs.core.windows.net/silver"
ADLS_TEST = "abfss://citibike@databricksjm.dfs.core.windows.net/test"

years = [fi.name.rstrip("/") for fi in dbutils.fs.ls(ADLS_BRONZE) if fi.isDir()]
#years = ["2015"]

# Load every year separately
for year_dir in years:
    print(f"********** PROCESSING: {year_dir} **********")
    whole_year_data = f"{ADLS_BRONZE}/{year_dir}"
    df = (
        spark.read
        .format("csv")
        .option("header", True)
        .load(whole_year_data)
    )
    # SAMPLE TEST - COMMENT LATER
    # df = df.sample(False, 0.02, seed=42).limit(200_000)

    # Check df format and normalize it by proper function
    if new_format(df):
        df = normalize_new(df, year_dir)
    else:
        df = normalize_old(df, year_dir)

    df = to_silver(df)
    
    print("Saving to silver...")
    (df.write
     .format("delta")
     .mode("replaceWhere", f"year ={int(year_dir)}")
     .partitionBy("year")
     .save(ADLS_SILVER)
     )
    print(f"********** FINISHED: {year_dir} **********\n")
